<a href="https://colab.research.google.com/github/davidwhogg/TargetSelection/blob/master/ipynb/selection_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selection Function
This notebook is part of the **TargetSelection** project.
It is designed to simulate a simple astronomical survey that
is being performed to measure simple properties of a set of stars.

## Authors
- **David W. Hogg** (NYU) (MPIA) (Flatiron)
- **Hans-Walter Rix** (MPIA)

## License
Copyright 2019 the authors. This code is released open source, under the *MIT License*.

In [0]:
import numpy as np
import matplotlib
import pylab as plt
!pip install emcee
import emcee
!pip install corner
import corner
%matplotlib inline
np.random.seed(17)

In [0]:
# Make a true Galaxy with the following properties:
scalelength = 3.0 # kpc
scaleheight = 0.25 # kpc
size = 32768 * 4 # desired final catalog size (ish)

# Initialize arrays
Rs = np.array([])
phis = np.array([])
zs = np.array([])
Ntrue = 0

# Fill arrays by repeated rejection sampling
while Ntrue < size:
  xs = 80. * (np.random.uniform(size=size) - 0.5)
  ys = 80. * (np.random.uniform(size=size) - 0.5)
  tzs = np.random.exponential(scaleheight, size=size)
  tzs[np.random.uniform(size=size) < 0.5] *= -1.
  tRs = np.sqrt(xs * xs + ys * ys)
  tphis = np.arctan2(ys, xs)
  keep = np.random.uniform(size=size) < np.exp(-tRs / scalelength)
  Rs = np.append(Rs, tRs[keep])
  phis = np.append(phis, tphis[keep])
  zs = np.append(zs, tzs[keep])
  Ntrue = len(Rs)

# Make absolute magnitudes, stupidly
Ms = 3. * np.random.uniform(size=Ntrue)

# Create a trivial structure to hold this all
class Truth(object):
  Rs = Rs
  phis = phis
  zs = zs
  vecs = np.vstack((Rs * np.cos(phis), Rs * np.sin(phis), zs)).T
  Ms = Ms
  N = Ntrue
  trueids = np.arange(N)

In [0]:
# Check that the Truth looks okay
plt.figure()
plt.scatter(Truth.Rs * np.cos(Truth.phis), Truth.zs, s= 0.01, c="k", alpha=0.5)
plt.axis("equal")
plt.figure()
plt.scatter(Truth.Rs * np.cos(Truth.phis), Truth.Rs * np.sin(Truth.phis), s= 0.01, c="k", alpha=0.5)
plt.axis("equal")

In [0]:
# Choose some observed field centers as unit vectors
Ntmp = 512
uvecs = np.random.normal(size=(Ntmp, 3))
uvecs /= np.sqrt(np.sum(uvecs * uvecs, axis=1))[:, None]

# Remove any that are too close to any others
# (self-avoid)
field_radius = 3.5 * np.pi / 180. # radians
conflict_matrix = np.sum(uvecs[:, None, :] * uvecs[None, :, :], axis=2) > np.cos(2. * field_radius)
conflict_matrix[np.triu_indices(Ntmp)] = False
good = np.sum(conflict_matrix, axis=1) == 0
uvecs = uvecs[good, :]

# Drop to M fields and make a trivial object
class Observer(object):
  vec = np.array([-8.022, 0., 0.]) # kpc ; should z not be zero??
  radius = field_radius # radians
  cos_radius = np.cos(field_radius)
  M = 128
  uvecs = uvecs[:M, :]
  fieldids = np.arange(M)
  completeness = 1. # placeholder

In [0]:
# Find the sources that are in the fields.
# Make a list "true_catalog" which is a set of field-object pairs.
vecs = Truth.vecs - Observer.vec[None, :] # vectors from Observer to sources
distances = np.sqrt(np.sum(vecs * vecs, axis=1)) # kpc
duvecs = vecs / distances[:, None]
true_catalog = []
n_infield = np.zeros(Observer.M)
for field in range(Observer.M):
  uvec = Observer.uvecs[field, :]
  infield = np.sum(duvecs * uvec[None, :], axis=1) > Observer.cos_radius
  n_infield[field] = np.sum(infield)
  true_catalog += [(field, trueid, duvecs[trueid], distances[trueid]) for trueid in Truth.trueids[infield]]
print(true_catalog[0:8])

In [0]:
# Now it is time to "observe" the objects.
# This will happen in stages.

# First, measure noisy magnitudes and noisy distances and cut on the measurements.
# This will make the "imaging_catalog".
brightlim = 8. # mag
faintlim = 15. # mag
magerr = 0.02 # mag

imaging_catalog = []
for field, trueid, uvec, distance in true_catalog:
  DMtrue = 5. * np.log10(distance / 0.01) # 0.01 kpc is 10 pc
  m = Truth.Ms[trueid] + DMtrue + magerr * np.random.normal()
  if m > brightlim and m < faintlim:
    imaging_catalog += [(field, trueid, uvec, m, DMtrue), ]
  
print(len(imaging_catalog))
print(imaging_catalog[0:8])
ifields = np.array([foo[0] for foo in imaging_catalog])
print(np.bincount(ifields, minlength=Observer.M))

In [0]:
# Second, go to each field, select AT MOST 32 per field, and take spectra.
# These spectra deliver noisy measurements of DM
# This will make the "spectroscopic_catalog"
DMerr = 0.2 # mag
max_per_field = 32

spectroscopic_catalog = []
number_per_field = np.zeros(Observer.M).astype(int)
for field, trueid, uvec, m, DMtrue in imaging_catalog:
  if number_per_field[field] < max_per_field:
    number_per_field[field] += 1
    DM = DMtrue + DMerr * np.random.normal()
    spectroscopic_catalog += [(field, trueid, uvec, m, DM)]
    
print(len(spectroscopic_catalog))
print(spectroscopic_catalog[:10])

In [0]:
# Compute spectroscopic completeness
# Note the issue that if there are zero in the imaging, the spectroscopic 
# completeness is 1.0 not 0.0!
ifields = np.array([foo[0] for foo in imaging_catalog])
sfields = np.array([foo[0] for foo in spectroscopic_catalog])
numerator   = np.clip(np.bincount(sfields, minlength=Observer.M), 0.5, np.inf)
denominator = np.clip(np.bincount(ifields, minlength=Observer.M), 0.5, np.inf)
spec_completeness = numerator / denominator
print(spec_completeness)
Observer.completeness = spec_completeness

In [0]:
# Make a trivial object to hold the spectroscopic catalog
class Catalog(object):
  N = len(spectroscopic_catalog)
  fields  = np.array([foo[0] for foo in spectroscopic_catalog])
  trueids = np.array([foo[1] for foo in spectroscopic_catalog])
  uvecs   = np.array([foo[2] for foo in spectroscopic_catalog])
  ms      = np.array([foo[3] for foo in spectroscopic_catalog])
  DMs     = np.array([foo[4] for foo in spectroscopic_catalog])
print(Catalog.N, Catalog.fields.shape, Catalog.uvecs.shape, Catalog.DMs.shape)

In [0]:
# Check the catalog.
distances = 0.01 * 10. ** (0.2 * Catalog.DMs) # kpc
poss = distances[:, None] * Catalog.uvecs
plt.figure()
plt.scatter(poss[:, 0], poss[:, 2], s= 0.1, c="k", alpha=0.9)
plt.axis("equal")
plt.xlabel(r"$x$ (kpc)")
plt.ylabel(r"$z$ (kpc)")
plt.figure()
plt.scatter(poss[:, 0], poss[:, 1], s= 0.1, c="k", alpha=0.9)
plt.axis("equal")
plt.xlabel(r"$x$ (kpc)")
plt.ylabel(r"$y$ (kpc)")

def plot_fields(Ob, lw=0.5, alpha=0.3, alphal=0.9, zorder=0):
  for uvec, comp in zip(Ob.uvecs, Ob.completeness):
    # make orthogonal vectors; slightly unsafe
    v1 = np.cross(uvec, [0., 0., 1.])
    v1 /= np.sqrt(np.dot(v1, v1))
    v2 = np.cross(uvec, v1)
    thetas = np.arange(0., 2. * np.pi + 0.00001, 0.001 * np.pi)
    uvecs = uvec[None, :] + np.tan(Ob.radius) \
          * (np.cos(thetas)[:, None] * v1[None, :]
           + np.sin(thetas)[:, None] * v2[None, :])
    uvecs /= np.sqrt(np.sum(uvecs * uvecs, axis=1))[:, None]
    ls = np.arctan2(uvecs[:, 1], uvecs[:, 0])
    bs = np.arcsin(uvecs[:, 2])
    if np.max(ls) - np.min(ls) > 1.9 * np.pi:
      ls[ls < 0.] += 2. * np.pi
      plt.fill(ls, bs, c="r", alpha=alpha*comp, zorder=zorder, lw=0)
      plt.plot(ls, bs, "r-",  alpha=alphal,     zorder=zorder, lw=lw)
      ls -= 2. * np.pi
    plt.fill(  ls, bs, c="r", alpha=alpha*comp, zorder=zorder, lw=0)
    plt.plot(  ls, bs, "r-",  alpha=alphal,     zorder=zorder, lw=lw)

plt.figure(figsize=(12, 6))
plt.scatter(np.arctan2(Catalog.uvecs[:, 1], Catalog.uvecs[:, 0]),
            np.arcsin(Catalog.uvecs[:, 2]), s=0.5, c="k", alpha=0.9)
plot_fields(Observer, zorder=-np.inf)
plt.xlabel(r"$\ell$ (rad)")
plt.ylabel(r"$b$ (rad)")
plt.xlim(-np.pi, np.pi)
plt.ylim(-0.5 * np.pi, 0.5 * np.pi)